In [6]:
%pylab inline
import rlcompleter, readline
readline.parse_and_bind('tab: complete')
import os
import csv
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from pyspark import SparkContext
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
from pyspark.sql.functions import monotonicallyIncreasingId
from pyspark.ml.feature import OneHotEncoder,StringIndexer

Populating the interactive namespace from numpy and matplotlib


In [7]:
data = sc.textFile('../dac/train.txt', 10000)
sample = sc.textFile('../dac_sample/dac_sample.txt').map(lambda x: x.split('\t'))
sample = sample.map(lambda x: x[1:])
split = data.map(lambda x: x.split('\t'))
label = split.map(lambda x: x[0])
entire_train = split.map(lambda x: x[1:])

In [8]:
train10M, train38M = entire_train.randomSplit([10,38])
train5M, validation2M, test3M = train10M.randomSplit([5,2,3])

In [9]:
df_train = sqlContext.createDataFrame(train5M, ['C'+str(i) for i in range(39)])
df_sample = sqlContext.createDataFrame(sample, ['C'+str(i) for i in range(39)])

#Convert String Column to Numeric

In [10]:
def changeDataType(df):
    new = df    
    for i in range(13):
        new = new.withColumn('C'+str(i), new['C'+str(i)].cast('int'))
    return new

In [34]:
new_sample = changeDataType(df_sample)
new_train = changeDataType(df_train)

#Statistics

In [12]:
def caculateMean(df):
    means = []
    for i in range(13):
        means.append(new_sample.groupBy().mean('C'+str(i)).collect()[0][0])
    return means

In [13]:
means = caculateMean(new_sample)
new_sample

DataFrame[C0: int, C1: int, C2: int, C3: int, C4: int, C5: int, C6: int, C7: int, C8: int, C9: int, C10: int, C11: int, C12: int, C13: string, C14: string, C15: string, C16: string, C17: string, C18: string, C19: string, C20: string, C21: string, C22: string, C23: string, C24: string, C25: string, C26: string, C27: string, C28: string, C29: string, C30: string, C31: string, C32: string, C33: string, C34: string, C35: string, C36: string, C37: string, C38: string]

In [14]:
def calculateStd(df):
    stds = []
    for i in range(13):
        std = float(new_sample.describe('C'+str(i)).collect()[2][1])
        stds.append(std)
    return stds

In [15]:
stds = calculateStd(new_sample)
new_sample

DataFrame[C0: int, C1: int, C2: int, C3: int, C4: int, C5: int, C6: int, C7: int, C8: int, C9: int, C10: int, C11: int, C12: int, C13: string, C14: string, C15: string, C16: string, C17: string, C18: string, C19: string, C20: string, C21: string, C22: string, C23: string, C24: string, C25: string, C26: string, C27: string, C28: string, C29: string, C30: string, C31: string, C32: string, C33: string, C34: string, C35: string, C36: string, C37: string, C38: string]

#Fill in Numeric Missing Value with Mean
1) fill in median for numeric data

In [16]:
def fillMissing(df):
    cols_name = ['C'+str(i) for i in range(13)]
    fill_data = means
    dic = dict(zip(cols_name,fill_data))
    #print dic
    new = df.fillna(dic)
    return new

In [17]:
filled_numeric = fillMissing(new_sample)

#Histogram

In [44]:
def drawHistogram(cols, i):
    col = cols[i]
    count = col.histogram(col.distinct().count())
    return count

In [43]:
counts = []

In [ ]:
print "drawing histogram"
for i in range(13):
    print str(i) + "th col"
    new = drawHistogram(cols, i)
    counts.append(new)
    plt.bar(new[0][1:], np.log10(new[1]), width = 1)
    plt.savefig("../images/log_col_"+str(i)+".png")

#Standardization

In [71]:
def standardize(cols, i):
    col = cols[i]
    mean = col.mean()
    std = col.stdev()
    new = col.map(lambda x : ((x-mean)/std))
    return new

In [72]:
cols_sample = [filled_numeric.select('C' + str(i)).rdd.flatMap(lambda x:x) for i in range(13)]
cols = [new_train.select('C' + str(i)).rdd.flatMap(lambda x:x) for i in range(13)]

In [73]:
standardize_cols = []
standardize_cols_sample = []

In [74]:
print "start standardizing"
for i in range(13):
    print str(i) + "th col"
    new = standardize(cols_sample, i)
    standardize_cols_sample.append(new)

start standardizing
0th col
1th col
2th col
3th col
4th col
5th col
6th col
7th col
8th col
9th col
10th col
11th col
12th col


In [76]:
a = standardize_cols_sample[0]


AttributeError: 'PipelinedRDD' object has no attribute 'T'

#Encoding

In [64]:
filled_numeric.withColumn('C0', standardize_cols_sample[0])

AssertionError: col should be Column

In [65]:
filled_numeric

DataFrame[C0: int, C1: int, C2: int, C3: int, C4: int, C5: int, C6: int, C7: int, C8: int, C9: int, C10: int, C11: int, C12: int, C13: string, C14: string, C15: string, C16: string, C17: string, C18: string, C19: string, C20: string, C21: string, C22: string, C23: string, C24: string, C25: string, C26: string, C27: string, C28: string, C29: string, C30: string, C31: string, C32: string, C33: string, C34: string, C35: string, C36: string, C37: string, C38: string]